In [1]:
import pandas as pd
import numpy as np

from preproc3 import encode, split, binarize, shuffle_in_unison, scale
from imblearn.over_sampling import SMOTE

from collections import Counter
from sklearn.model_selection import cross_val_score
from alg7 import model

In [2]:
DATA='ugrin2020-vehiculo-usado-multiclase/'
TRAIN=DATA+'train.csv'
TEST=DATA+'test.csv'

RESULTS='results/'

NOMBRE=DATA+'nombre.csv'
CIUDAD=DATA+'ciudad.csv'
COMBUSTIBLE=DATA+'combustible.csv'
TIPO_MARCHAS=DATA+'tipo_marchas.csv'

In [3]:
train = pd.read_csv(TRAIN) # Cargo datos de entrenamiento
test = pd.read_csv(TEST) # Cargo datos de test

# Eliminamos el campo id ya que no se debe usar para predecir
test_ids = test['id']
del test['id']
del train['id']

# Cambiamos el nombre a la columna Año para poder manejarla correctamente
train.rename(columns = {'Año':'Anio'}, inplace = True)
test.rename(columns = {'Año':'Anio'}, inplace = True)

In [4]:
print(model)

GradientBoostingClassifier(n_estimators=500, random_state=25)


In [5]:
train.shape[0]

4819

In [6]:
val=train[train.Descuento.notna()]
train=train[train.Descuento.isna()]
val.dropna(inplace=True)
descuentos=val.Descuento

del train['Descuento']
del test['Descuento']
del val['Descuento']
train.dropna(inplace=True)

In [8]:
val

,Nombre,Ciudad,Anio,Kilometros,Combustible,Tipo_marchas,Mano,Consumo,Motor_CC,Potencia,Asientos,Precio_cat
14,Audi A4 35 TDI Premium Plus,E,2014.0,28469.0,Diesel,Automatic,First,18.25 kmpl,1968 CC,187.74 bhp,5.0,5
31,Maruti Dzire VDI,I,2015.0,28193.0,Diesel,Manual,First,28.4 kmpl,1248 CC,73.75 bhp,5.0,3
32,Mahindra Bolero SLE,F,2012.0,75000.0,Diesel,Manual,First,15.96 kmpl,2523 CC,62.1 bhp,7.0,3
39,Tata Zest Quadrajet 1.3 75PS XE,C,2018.0,45000.0,Diesel,Manual,First,22.95 kmpl,1248 CC,74 bhp,5.0,3
42,Tata Zest Revotron 1.2T XE,J,2017.0,47000.0,Petrol,Manual,Second,17.57 kmpl,1193 CC,88.7 bhp,5.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
4776,Honda Jazz VX Diesel,D,2016.0,14980.0,Diesel,Manual,First,27.3 kmpl,1498 CC,98.6 bhp,5.0,4
4784,Mercedes-Benz GLE 350d,K,2016.0,28541.0,Diesel,Automatic,First,11.57 kmpl,2987 CC,254.79 bhp,5.0,5
4788,Maruti Vitara Brezza ZDi Plus Dual Tone,B,2016.0,20000.0,Diesel,Manual,First,24.3 kmpl,1248 CC,88.5 bhp,5.0,4
4793,Honda BRV i-VTEC V CVT,B,2017.0,18000.0,Petrol,Automatic,First,16.0 kmpl,1497 CC,117.3 bhp,7.0,4


In [9]:
train, val = encode(train, val)
train, label, meh = split(train)
val, val_label, meh = split(val)
train, val = binarize(train, val)
train, label = SMOTE(random_state=25).fit_resample(train, label)
shuffle_in_unison(train, label)
print(train.shape)
print(val.shape)
train, val = scale(train,val)

(8050, 52)
(571, 52)


In [10]:
scores=cross_val_score(model, train, label)
print(scores)
print(np.mean(scores))

[0.90496894 0.90993789 0.91863354 0.91925466 0.92111801]
0.9147826086956522


In [11]:
model.fit(train, label)
pred = model.predict(val)

In [25]:
dif = pred-val_label # Si sale positivo es que me he pasado
np.mean([i for i in dif if i!=0])

-0.00980392156862745

In [33]:
diferencias=pd.DataFrame(np.hstack((val_label.reshape(-1,1),pred.reshape(-1,1),dif.reshape(-1,1),np.array(descuentos).reshape(-1,1))),columns=['Cat','Pred','Dif','Desc'])
diferencias

,Cat,Pred,Dif,Desc
0,5.0,5.0,0.0,53.14
1,3.0,3.0,0.0,8.36
2,3.0,3.0,0.0,10.32
3,3.0,3.0,0.0,8.34
4,3.0,3.0,0.0,6.62
...,...,...,...,...
566,4.0,4.0,0.0,10.81
567,5.0,5.0,0.0,95.13
568,4.0,4.0,0.0,11.98
569,4.0,4.0,0.0,15.02


In [36]:
diferencias.groupby('Dif').count()

,Cat,Pred,Desc
Dif,,,
-2.0,1,1,1
-1.0,52,52,52
0.0,469,469,469
1.0,47,47,47
2.0,1,1,1
4.0,1,1,1


In [35]:
diferencias.groupby('Dif').mean()

,Cat,Pred,Desc
Dif,,,
-2.0,3.000000,1.000000,8.120000
-1.0,3.826923,2.826923,12.164231
0.0,3.840085,3.840085,20.769446
1.0,2.808511,3.808511,16.253617
2.0,3.000000,5.000000,52.460000
4.0,1.000000,5.000000,1.360000


In [43]:
model.fit(train,label)
# Ahora predecimos
predict = model.predict(test)
predict = list(map(int,predict))
# Generamos 
df_result = pd.DataFrame({'id': test_ids, 'Precio_cat': predict})
df_result.to_csv(RESULTS+"try11.csv", index=False)